# 연관규칙 평가척도

1. 지지도(support): 아이템 집합이 전체 트랜잭션 데이터에서 발생한 비율
    - S(A->B) = N(A, B)(트랜잭션 데이터에서 A와 B의 동시 출현 횟수) / n(트랜잭션 데이터 크기)
2. 신뢰도(confidence): 부모 아이템 집합이 등장한 트랜잭션 데이터에서 자식 아이템 집합이 발생한 비율
    - C(A->B) = N(A, B)(트랜잭션 데이터에서 A와 B의 동시 출현 횟수) / N(A)(트랜잭션 데이터에서 A의 출현 횟수)


- 지지도와 신뢰도가 높은 연관규칙을 좋은 규칙이라고 함
- 지지도에 대한 Apriori 원리
    - S(A->B)가 최소 지지도 이상이면 이 규칙을 빈발하다고 함
    - 아이템 집합의 지지도가 최소 지지도 이상이면 이 집합을 빈발하다고 함
    - 어떤 아이템 집합이 빈발하면, 이 아이템의 부분 집합도 빈발함
    - 후보 규칙 생성
        - Apriori 원리를 사용하여 모든 최대 빈발 아이템 집합을 찾은 후, 후보 규칙을 모두 생성함
        - 최대 빈발 아이템 집합: 최소 지지도 이상이면서, 이 집합의 모든 모집합이 빈발하지 않는 집합
- 신뢰도에 대한 Apriori 원리

# 1. 연관규칙 탐색

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Instacart Market Basket Analysis.csv')
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [3]:
df['product_id'].value_counts(normalize = True).head(100)

24852    0.013524
13176    0.011180
21137    0.007868
21903    0.007066
47626    0.005875
           ...   
8193     0.001024
9387     0.000996
37687    0.000984
20995    0.000983
6184     0.000976
Name: product_id, Length: 100, dtype: float64

In [4]:
product_list_per_order = df.groupby('order_id')['product_id'].apply(list)
product_list_per_order

order_id
1          [49302, 11109, 10246, 49683, 43633, 13176, 472...
36         [39612, 19660, 49235, 43086, 46620, 34497, 486...
38         [11913, 18159, 4461, 21616, 23622, 32433, 2884...
96         [20574, 30391, 40706, 25610, 27966, 24489, 39275]
98         [8859, 19731, 43654, 13176, 4357, 37664, 34065...
                                 ...                        
3421049           [40800, 17706, 33424, 17299, 26800, 34243]
3421056                    [5750, 9340, 21709, 16475, 12432]
3421058    [15629, 4347, 34466, 6244, 6858, 30316, 35578,...
3421063                         [49235, 13565, 14233, 35548]
3421070                                 [35951, 16953, 4724]
Name: product_id, Length: 131209, dtype: object

In [5]:
# 구매 기록 데이터 -> one hot encoding
## from mlxtend.preprocessing import TransactionEncoder
## 연관규칙 탐사에 적절하게 거래 데이터 구조를 바꾸기 위한 라이브러리

from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()     # 인스턴스 생성

## fit(data).transform(data): data를 각 아이템의 출현 여부를 갖는 ndarray 형식으로 변환
one_hot_df = encoder.fit(product_list_per_order).transform(product_list_per_order)     # ndarray 형식
one_hot_df = pd.DataFrame(one_hot_df, columns = encoder.columns_)
one_hot_df.head()

,1,2,3,4,5,7,8,9,10,11,...,49677,49678,49679,49680,49681,49682,49683,49686,49687,49688
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
from mlxtend.frequent_patterns import *

## apriori(df, min_support): df: one hot encoding 형태의 데이터 프레임, min_support: 최소 지지도
frequent_item_df = apriori(one_hot_df, min_support = 0.003)     # 0.3% 이상 구매한 상품만 대상으로 함

In [ ]:
## association_rules(frequent_dataset, metric, min_threshold): frequent_dataset에서 찾은 연관 규칙을 데이터 프레임 형태로 반환
### metric: 연관규칙을 필터링하기 위한 유용성 척도(default = confidence), min_threshold: 지정한 metric의 최소 기준치
result = association_rules(frequent_item_df, metric = 'confidence', min_threshold = 0.1)

In [ ]:
result[['antecedents', 'consequents', 'support', 'confidence']].sort_values(by = 'confidence', ascending = False).to_csv('연관규칙탐색결과.csv', index = False)

In [ ]:
result.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(231),(12121),0.010177,0.141794,0.003164,0.310924,2.192786,0.001721,1.245445
1,(1131),(7463),0.009536,0.014496,0.003079,0.322870,22.273265,0.002941,1.455413
2,(7463),(1131),0.014496,0.009536,0.003079,0.212389,22.273265,0.002941,1.257556
3,(2256),(10739),0.028008,0.073976,0.004191,0.149618,2.022528,0.002119,1.088951
4,(2256),(12121),0.028008,0.141794,0.008167,0.291603,2.056523,0.004196,1.211476


# 2. 빈발 시퀀스 탐색(시퀀스 데이터에 대한 연관규칙 탐색)

- 시퀀스 데이터: 각 요소가 (순서, 값) 형태로 구성된 데이터. 분석 시에 반드시 순서를 고려해야 함
- 예: 로그 데이터(고객 구매 기록, 고객 여정, 웹 서핑 기록)

### 순서를 고려한 연관규칙 탐사

- 시퀀스 데이터에 대한 연관규칙 탐사에 대해서는 A -> B와 B -> A가 다른 지지도를 갖기 때문에, 같은 항목 집합으로부터 규칙을 생성할 수 없음
- 신뢰도에 대한 apriori원리는 성립함
- 따라서 개별요소(이벤트)에 다른 요소를 추가하는 방식으로 규칙을 아래와 같이 직접 찾아나가야 함
- 유니크한 요소 목록 추출 -> 빈발하는 단일 이벤트 추출 -> 이벤트 추가 및 지지도 계산 -> 최대 빈발 아이템 집합 탐색 -> 규칙 생성

### 동적 프로그래밍

- 원 문제를 작은 문제로 분할한 다음 점화식으로 만들어 재귀적인 형태로 원 문제를 해결하는 방식


In [6]:
import numpy as np

df = pd.read_csv('페이지내_사용자_이동.csv', encoding = 'cp949')
df.head()

,고객ID,방문 페이지,순서
0,0,페이지C,1
1,0,페이지E,2
2,0,페이지B,3
3,0,페이지F,4
4,0,페이지C,5


In [7]:
# 주문별 카트에 추가한 순서를 고려하기 위해 정렬 필요

df.sort_values(by = ['고객ID', '순서'], inplace = True)

In [8]:
page_set = df['방문 페이지'].unique()
page_set

array(['페이지C', '페이지E', '페이지B', '페이지F', '페이지D', '페이지J', '메인', '페이지G',
       '페이지A', '페이지I', '페이지H'], dtype=object)

In [9]:
page_sequence_per_order = df.groupby('고객ID')['방문 페이지'].apply(np.array)
page_sequence_per_order

고객ID
0     [페이지C, 페이지E, 페이지B, 페이지F, 페이지C, 페이지D, 페이지J, 메인,...
1     [페이지B, 메인, 메인, 페이지A, 페이지F, 페이지C, 페이지I, 페이지E, 페...
2     [페이지F, 페이지J, 페이지D, 페이지G, 페이지C, 메인, 페이지I, 페이지J,...
3     [페이지I, 페이지I, 페이지J, 페이지J, 페이지C, 페이지A, 페이지H, 페이지...
4     [페이지J, 페이지H, 페이지G, 페이지G, 페이지E, 페이지C, 페이지A, 페이지...
                            ...                        
95    [페이지D, 페이지H, 페이지I, 메인, 페이지B, 페이지E, 페이지F, 페이지G,...
96    [페이지E, 페이지I, 페이지E, 페이지G, 페이지A, 페이지E, 페이지D, 페이지...
97    [페이지B, 페이지B, 메인, 메인, 페이지B, 페이지F, 페이지J, 페이지D, 페...
98                               [메인, 페이지D, 페이지F, 페이지G]
99    [페이지E, 페이지H, 페이지B, 페이지D, 페이지B, 페이지A, 페이지I, 페이지...
Name: 방문 페이지, Length: 100, dtype: object

In [10]:
from itertools import product

def contain_pattern(record, pattern, L):
    output = False
    if set(record) & set(pattern) != set(pattern): # pattern에 포함된 모든 아이템 집합이 record에 포함된 아이템 집합에 속하지 않으면
        return False
    else:
        # 패턴에 속한 개별 아이템에 대한 위치를 미리 구하기
        pattern_index_list = [np.where(record == item)[0] for item in pattern]

        ## 가능한 모든 조합에서 위치 간 거리가 L이하면 True를 반환
        # record = [A, B, C, A, C, C], pattern = [A, B], L = 1
        # A의 위치: [0, 3], B의 위치: [1]
        # 가능한 모든 조합: [0, 1], [3, 1]
        # 가능한 모든 조합의 거리 차이: [1 - 0, 1 - 3] 중에 0번째 요소는 만족하므로 True

        for pattern_index in product(*pattern_index_list):
            distance = np.array(pattern_index)[1:] - np.array(pattern_index)[:-1]
            if sum((distance <= L) & (distance > 0)) == (len(pattern_index) - 1):
                output = True
                break

        return output

In [11]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support = 0.01, L = 1):
    queue = []
    maximum_frequent_sequence_item = []

    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 quene에 추가시킴
    for item in item_set:
        occurence = sequence_data.apply(contain_pattern, pattern = [item], L = L).sum()
        if occurence / len(sequence_data) >= min_support:
            queue.append([item])

    while queue:
        current_pattern = queue.pop()     # 맨 마지막 값 빼기
        check_maximum_frequent = True     # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set:
            occurence = sequence_data.apply(contain_pattern, pattern = current_pattern + [item], L = L).sum()
            if occurence / len(sequence_data) >= min_support:     # min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False
                queue.append(current_pattern + [item])

        if check_maximum_frequent and len(current_pattern) > 1:
            maximum_frequent_sequence_item.append(current_pattern)

    return maximum_frequent_sequence_item

In [15]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
    # 결과 초기화
    result = {'부모': [], '자식': [], '지지도': [], '신뢰도': []}

    for sequence_item in maximum_frequent_sequence_item:
        # A -> B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산
        co_occurence = sequence_data.apply(contain_pattern, pattern = sequence_item, L = L).sum()
        support = co_occurence / len(sequence_data)
        if co_occurence > min_support:
            for i in range(len(sequence_item)-1, 0, -1):# 한 아이템 집합에 대해, 부모의 크기를 1씩 줄여나가는 방식으로 부모와 자식 설정
                antecedent = sequence_item[:i]
                consequent = sequence_item[i:]
                antecedent_occurence = sequence_data.apply(contain_pattern, pattern = antecedent, L = L).sum()
                
                confidence = co_occurence / antecedent_occurence
                
                if confidence > min_confidence:
                    result['부모'].append(antecedent)
                    result['자식'].append(consequent)
                    result['지지도'].append(support)
                    result['신뢰도'].append(confidence)
                    
    return pd.DataFrame(result)

In [16]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(page_set, page_sequence_per_order, min_support = 0.02, L = 1)

In [17]:
result = generate_association_rules(maximum_frequent_sequence_item, page_sequence_per_order, 
                                    min_support = 0.01, min_confidence = 0, L = 1)
result.sort_values(by = ['지지도', '신뢰도'], ascending = False)

,부모,자식,지지도,신뢰도
160,[페이지B],[페이지J],0.07,0.155556
59,[페이지A],[페이지C],0.07,0.148936
109,[페이지J],[페이지D],0.07,0.142857
115,[페이지J],[페이지C],0.07,0.142857
75,[메인],[페이지H],0.06,0.150000
...,...,...,...,...
205,[페이지C],"[메인, 페이지A, 페이지J]",0.02,0.037037
207,[페이지C],"[메인, 페이지E]",0.02,0.037037
209,[페이지C],"[페이지J, 페이지I]",0.02,0.037037
214,[페이지C],"[페이지E, 메인]",0.02,0.037037


# 3. 빈발 시계열 패턴 (시계열 데이터에 대한 연관규칙 탐색)

- 시계열 데이터: 각 요소가 (시간, 값) 형태로 구성된 데이터로, 반드시 순서 및 시간을 고려해야 함

구분|시계열 데이터|시퀀스 데이터
-|-|-
인덱스|시간(순서)|순서
값|주로 연속형|주로 범주형

- 시계열 데이터는 연속형이라는 특징 때문에 패턴을 찾으려면 이산화가 필요함
- SAX(symbolic aggregate approximation)를 사용하면 시계열 데이터를 이산화 할 수 있음
- SAX는 윈도우 분할, 윈도우별 대표값 계산, 알파벳 시퀀스로 변환이라는 3단계로 구성됨

In [20]:
df = pd.read_excel('2018-20_코스피지수.xlsx')
df.head()

,날짜,종가
0,2020-07-17,2201.19
1,2020-07-16,2183.76
2,2020-07-15,2201.88
3,2020-07-14,2183.61
4,2020-07-13,2186.06


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      493 non-null    datetime64[ns]
 1   종가      493 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 7.8 KB


In [22]:
# 순서를 고려하기 위해 정렬 필요

df.sort_values(by = ['날짜'], inplace = True)

#### SAX 적용

In [23]:
def segmentation(x, w, a):      # x: time series sample, w: window size, a: alphabet size
    window_mean = x[:w * int(len(x) / w)].reshape(w, -1).mean(axis = 1)
    if len(x) % w != 0:
        window_mean = np.hstack([window_mean, x[w * int(len(x) / w):].mean()]) # 남겨진 부분은 그 부분만 평균해서 window_mean에 추가
        
    return window_mean

In [24]:
def find_break_points(wmv, a):     # wmv: window mean vector
    break_points = [np.quantile(wmv, (i + 1) / a) for i in range(a)]
    return break_points

In [25]:
def conversion_window(wv, break_points):     # wv: window mean vector
    alphabet_sequence = []
    for e in wv:
        alphabet = 65
        for bp in break_points:
            if e < bp:
                alphabet_sequence.append(chr(alphabet))
                break
            else:
                alphabet += 1
                
    return alphabet_sequence

In [26]:
KOSPI = df['종가'].values
window_mean_vector = segmentation(x = KOSPI, w = 100, a = 5)
window_mean_vector

array([2282.725     , 2284.32      , 2291.51      , 2293.9475    ,
       2273.465     , 2251.4475    , 2287.11      , 2310.595     ,
       2300.5325    , 2284.09      , 2304.1175    , 2331.6275    ,
       2316.5025    , 2219.9075    , 2154.9       , 2143.095     ,
       2046.02      , 2041.21      , 2084.465     , 2076.4525    ,
       2090.9       , 2071.75      , 2104.65      , 2104.07      ,
       2066.2725    , 2074.5325    , 2063.865     , 2026.8725    ,
       2016.58      , 2067.02      , 2108.655     , 2128.7975    ,
       2186.1475    , 2197.195     , 2199.6325    , 2210.5925    ,
       2229.58      , 2200.03      , 2154.2325    , 2159.345     ,
       2179.7725    , 2156.5575    , 2153.5575    , 2207.5025    ,
       2223.96      , 2237.7925    , 2213.585     , 2197.4575    ,
       2188.5175    , 2092.725     , 2067.995     , 2057.7525    ,
       2038.79      , 2061.4175    , 2098.095     , 2097.        ,
       2127.005     , 2127.1075    , 2114.1275    , 2071.3925 

In [27]:
bps = find_break_points(window_mean_vector, 5)
bps

[2064.98, 2090.9, 2154.2325, 2216.0199999999995, 2331.6275]

In [29]:
alphabet_sequence = conversion_window(window_mean_vector, bps)
print(alphabet_sequence)

['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'D', 'C', 'A', 'A', 'B', 'B', 'C', 'B', 'C', 'C', 'B', 'B', 'A', 'A', 'A', 'B', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'D', 'D', 'D', 'D', 'D', 'C', 'D', 'E', 'E', 'D', 'D', 'D', 'C', 'B', 'A', 'A', 'A', 'C', 'C', 'C', 'C', 'C', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'B', 'B', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'D', 'C', 'E', 'E', 'D', 'B', 'A', 'A']


#### 패턴 찾기

In [30]:
# 문자열로 변환(하나의 리스트만 대상으로 하므로 이런 식으로 변환. 단 L = 1로 한정)

alphabet_sequence = '-'.join(alphabet_sequence)
alphabet_sequence

'E-E-E-E-E-E-E-E-E-E-E-E-E-D-C-A-A-B-B-C-B-C-C-B-B-A-A-A-B-C-C-D-D-D-D-E-D-D-D-D-D-C-D-E-E-D-D-D-C-B-A-A-A-C-C-C-C-C-B-B-B-B-A-A-A-A-A-A-A-A-B-B-B-A-A-B-B-B-C-C-C-C-C-B-B-C-D-D-D-D-E-E-D-C-E-E-D-B-A-A'

In [34]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support = 0.01, L = 1):
    queue = []
    maximum_frequent_sequence_item = []
    
    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템드람ㄴ queue에 추가시킴
    for item in item_set:
        # 문자열이므로 count 사용
        occurence = sequence_data.count(item)
        if occurence / (sequence_data.count('-') + 1) >= min_support:
            queue.append(item)
            
    while queue:
        current_pattern = queue.pop()     # 맨 마지막 값 빼기
        check_maximum_frequent = True     # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set:
            occurence = sequence_data.count(current_pattern + '-' + item)
            if occurence / (sequence_data.count('-') + 1) >= min_support:     # min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False
                queue.append(current_pattern + '-' + item)
                
        if check_maximum_frequent:
            maximum_frequent_sequence_item.append(current_pattern)
            
    return maximum_frequent_sequence_item

In [35]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
    # 결과 초기화
    result = {'부모': [], '자식': [], '지지도': [], '신뢰도': []}
    
    for sequence_item in maximum_frequent_sequence_item:
        # A -> B에서 A, B를 모두 포함하는 가짓수 co_occurence 계산
        co_occurence = sequence_data.count(sequence_item)
        support = co_occurence / (sequence_data.count('-') + 1)
        if co_occurence > min_support:
            # 한 아이템 집합에 대해, 부모의 크기를 1씩 줄여나가는 방식으로 부모와 자식 설정
            for i in range(sequence_item.count('-') - 1, 0, -1): 
                antecedent = '-'.join(sequence_item.split('-')[:i])
                consequent = '-'.join(sequence_item.split('-')[i:])
                antecedent_occurence = sequence_data.count(antecedent)
                
                confidence = co_occurence / antecedent_occurence
                if confidence > min_confidence:
                    result['부모'].append(antecedent)
                    result['자식'].append(consequent)
                    result['지지도'].append(support)
                    result['신뢰도'].append(confidence)
                    
    return pd.DataFrame(result)

In [36]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item([chr(65 + i) for i in range(5)], 
                                                                     alphabet_sequence, min_support = 0.02, L = 1)

In [37]:
result = generate_association_rules(maximum_frequent_sequence_item, alphabet_sequence, min_support = 0.02, min_confidence = 0.1, L = 1)
result = result.sort_values(by = ['지지도', '신뢰도'], ascending = False)
result

,부모,자식,지지도,신뢰도
31,B-A,A-A,0.03,0.600000
22,C-C,B-B,0.03,0.500000
23,C,C-B-B,0.03,0.150000
24,C,B-B,0.03,0.150000
25,B,C-C,0.03,0.150000
26,B,B-C,0.03,0.150000
32,B,A-A-A,0.03,0.150000
10,C-D-D-D,D-E,0.02,1.000000
11,C-D-D,D-D-E,0.02,1.000000
13,C-C-C-C-C,B-B,0.02,1.000000
